
---

## 🔹 1. `MAX(CASE WHEN ... THEN ... ELSE NULL END)`

* `MAX` is **aggregate**.
* You usually use it in a `GROUP BY` or across partitions.
* Since `MAX` ignores `NULL`, this pattern is often used to **pick a single value out of rows**.

Example:

```sql
SELECT customer_id,
       MAX(CASE WHEN rnk = 1 THEN product_id ELSE NULL END) AS top_product
FROM Transactions
GROUP BY customer_id;
```

* Here, for each `customer_id`, only the row with `rnk=1` contributes a value.
* `MAX` returns that value (since there’s only one non-NULL per group).
* If **no row has `rnk=1`**, result is `NULL`.

---

## 🔹 2. `COALESCE(CASE WHEN ... THEN ... ELSE NULL END)`

* `COALESCE` is **row-level**, not aggregate.
* It just evaluates the `CASE` for that single row.
* If condition is true → returns `something`.
* Else → returns `NULL`.

Example:

```sql
SELECT customer_id,
       COALESCE(CASE WHEN rnk = 1 THEN product_id ELSE NULL END, -1) AS maybe_top
FROM Transactions;
```

* Here, it doesn’t “pick across rows.”
* It only says: for this row → if `rnk=1`, show product_id, otherwise `-1`.
* You’ll still get multiple rows per customer, unless you aggregate.

---

## 🔹 3. Key Difference

| Aspect             | `MAX(CASE...)`                          | `COALESCE(CASE...)`                                          |
| ------------------ | --------------------------------------- | ------------------------------------------------------------ |
| **Scope**          | Across rows (aggregate)                 | Per row (scalar)                                             |
| **NULL handling**  | Ignores NULLs, picks non-NULL if exists | Returns value for that row, else NULL (or fallback if given) |
| **Usage**          | Used to pivot/aggregate values          | Used to substitute NULL in row-level expression              |
| **Typical Output** | One row per group (with `GROUP BY`)     | One row per input row                                        |

---

## 🔹 4. Teaching Example

Say we have:

| customer_id | product_id | rnk |
| ----------- | ---------- | --- |
| 1           | A          | 1   |
| 1           | B          | 2   |
| 2           | C          | 2   |

👉 Using `MAX`:

```sql
SELECT customer_id,
       MAX(CASE WHEN rnk=1 THEN product_id END) AS top_product
FROM T
GROUP BY customer_id;
```

Result:

| customer_id | top_product |
| ----------- | ----------- |
| 1           | A           |
| 2           | NULL        |

👉 Using `COALESCE`:

```sql
SELECT customer_id,
       COALESCE(CASE WHEN rnk=1 THEN product_id END, 'NONE') AS maybe_top
FROM T;
```

Result:

| customer_id | maybe_top |
| ----------- | --------- |
| 1           | A         |
| 1           | NONE      |
| 2           | NONE      |

---

✅ **Interview takeaway:**

* `MAX(CASE...)` is for **aggregation (pick one value per group)**.
* `COALESCE(CASE...)` is for **row-level substitution (replace NULL)**.

---
